## Demo 2 the use of PyTorch.ipynb

    - Simple demo of PyTorch for creating shallow MLP Neural Net to solve 2 toy problems:
        * Regression Problem: Fitting a Noisy Sine function
        * Iris Classification Problem

### Define the imports

In [3]:
import torch
import torch.utils.data as Data
import numpy as np
import sklearn.datasets as dat
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl

%matplotlib inline


### Define Toy problems

In [4]:
class ToyProblems(object):
    @staticmethod
    def getDataAsBatch(X, y, batch_size):
        torch_dataset = Data.TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
        loader = Data.DataLoader(
            dataset=torch_dataset,      # torch TensorDataset format
            batch_size=batch_size,      # mini batch size
            shuffle=True,               # random shuffle for training
            num_workers=2,              # subprocesses for loading data
        )
        X_train_batch, y_train_batch = [], []
        for i, (batch_x, batch_y) in enumerate(loader):
            X_train_batch.append(batch_x)
            y_train_batch.append(batch_y)
        return (X_train_batch, y_train_batch)
         
    @staticmethod
    def toyRegressionProblemData(data_points = 1000, noise_value = 0.2, period = 3, train_batch_size = 8):
        """
        Mocks-up the Neural Net Noisy Sine Function Regression Problem Data
        """
        n_data = data_points
        X = np.linspace(0, 1, n_data).reshape(n_data,1)
        np.random.seed(10)
        y = np.sin(2*period*np.pi*X) + noise_value*np.random.rand(n_data).reshape(n_data,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        X_train_batch, y_train_batch = ToyProblems.getDataAsBatch(X_train, y_train, train_batch_size)
        toy_data = {}
        toy_data['X_train'] = torch.from_numpy(X_train).float()
        toy_data['y_train'] = torch.from_numpy(y_train).float()   
        toy_data['X_train_batch'] = X_train_batch
        toy_data['y_train_batch'] = y_train_batch
        toy_data['X_test'] = torch.from_numpy(X_test).float()
        toy_data['y_test'] = torch.from_numpy(y_test).float()        
        return toy_data
    
    @staticmethod
    def toyClassificationProblemDataOne():
        """
        Mocks-up the Neural Net Classification problem data - it is based on a sub-set of the classic Iris data
        Returns the Feature (X) / Response (y) data and the number of classes
        """
        np.random.seed(10)
        n_data = 20
        iris = dat.load_iris()
        X, y = iris.data, iris.target
        X, y = shuffleDataPairs(X, y)
        X, y = X[:n_data], y[:n_data]
        n_classes = len(iris.target_names)
        return (X, y, n_classes)
    

### Specify the Multi Layer Neural Network

In [5]:
class LayerStruct(object):
    def __init__(self, start_nodes, end_nodes, activation):
        self.start_nodes = start_nodes
        self.end_nodes = end_nodes
        self.activation = activation

class MultiLayerNet(torch.nn.Module):
    def __init__(self, layer_structs):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(MultiLayerNet, self).__init__()
        self.layer_structs = layer_structs
        self.num_layers = len(self.layer_structs)
        #self.layers = [None] * self.num_layers
        self.layers = torch.nn.ModuleList()
        for i, layer_struct in enumerate(self.layer_structs):
            print("layer_struct.start_nodes: {0}, layer_struct.end_nodes: {1}".format(layer_struct.start_nodes, layer_struct.end_nodes))
            self.layers.append(torch.nn.Linear(layer_struct.start_nodes, layer_struct.end_nodes))
        #self.layers = torch.nn.ModuleList(temp)
        

    def forward(self, X):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h = X
        num_layers = self.num_layers - 1
        for i in range(num_layers):
            h = self.layers[i](h)
            if self.layer_structs[i].activation == 'R':
                h = h.clamp(min=0)
                #print("i: {}".format(i))
        y_pred = self.layers[num_layers](h)
        return y_pred


### Specify the call Neural Net to solve problems 

In [6]:
def runNeuralNet(
    layer_structs, 
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    is_batch = False):
    # Construct our model by instantiating the class defined above
    model = MultiLayerNet(layer_structs)
    print("Model:\n{}".format(model))
    #print("model.parameters: {}".format(list(model.parameters())))
    
    # Define results container
    results = []
    
    # Construct our loss function and an Optimizer. The call to model.parameters()
    # in the SGD constructor will contain the learnable parameters of the two
    # nn.Linear modules which are members of the model.
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.SGD(list(model.parameters()), lr=1e-4)
    for t in range(1000):
        results = executeEpochAsBatch(criterion,optimizer,model, X_train, y_train, results, t, is_batch_run=is_batch)
                
    # Now use the odel to predict the outcome of unseen data
    y_test_hat = model.forward(X_test)
    return (results, y_test_hat)

def executeEpoch(criterion,optimizer,model, X_train, y_train, t, batch_i=None):
    # Forward pass: Compute predicted y by passing x to the model
    #print("X_train.shape: {}".format(X_train.shape))
    y_pred = model(X_train)

    # Compute and print loss
    loss = criterion(y_pred, y_train)
    result_i = (t, loss)
    if t % 50 == 0:
        if batch_i != None:
            print("Current result for iteration {0} and batch {1} is {2}".format(result_i[0], batch_i, result_i[1]))
        else:
            print("Current result for iteration {0} is {1}".format(result_i[0], result_i[1]))

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return result_i

def executeEpochAsBatch(criterion,optimizer,model, X_train, y_train, results, t, is_batch_run=False):
    if is_batch_run:
        num_batch = len(X_train)
        for i in range(num_batch):
            result = executeEpoch(criterion,optimizer,model, X_train[i], y_train[i], t, i)
            results.append(result)
    else:
        result = executeEpoch(criterion,optimizer,model, X_train, y_train, t)
        results.append(result)
    return results

def solveNoisySineProblem():
    batch_size = 64
    toy_data = ToyProblems.toyRegressionProblemData(train_batch_size = batch_size)
    X_train = toy_data['X_train']
    y_train = toy_data['y_train']
    X_train_batch = toy_data['X_train_batch']
    y_train_batch = toy_data['y_train_batch']
    X_test = toy_data['X_test']
    y_test = toy_data['y_test']
    num_inputs = X_train.shape[1]
    num_l1_hidden = 10
    num_l2_hidden = 10
    num_outputs = y_train.shape[1]
    layer_structs = [
        LayerStruct(num_inputs, num_l1_hidden, 'R'),
        LayerStruct(num_l1_hidden, num_l2_hidden,'R'),
        LayerStruct(num_l2_hidden, num_outputs,'L')
    ]
    results, y_test_hat = runNeuralNet(
    layer_structs, 
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    is_batch=True)
    
def solveBasicRandomProblem():
    N, inputs, outputs = 64, 1000, 10
    X = np.random.randn(N, inputs)
    y = np.random.randn(N, outputs)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    num_inputs = X_train.shape[1]
    num_l1_hidden = 100
    #num_l2_hidden = 10
    num_outputs = y_train.shape[1]
    layer_structs = [
        LayerStruct(num_inputs, num_l1_hidden, 'R'),
        #LayerStruct(num_l1_hidden, num_l2_hidden,'R'),
        LayerStruct(num_l1_hidden, num_outputs,'L')
    ]
    X_train = torch.from_numpy(X_train).float()
    y_train = torch.from_numpy(y_train).float()
    X_test = torch.from_numpy(X_test).float()
    y_test = torch.from_numpy(y_test).float()
#     X_train = torch.randn(N, inputs)
#     y_train = torch.randn(N, outputs)
#     X_test = torch.randn(N, inputs)
#     y_test = torch.randn(N, outputs)
    results, y_test_hat = runNeuralNet(
    layer_structs, 
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    is_batch=False)
    
    

In [ ]:
solveNoisySineProblem()
#solveBasicRandomProblem()

layer_struct.start_nodes: 1, layer_struct.end_nodes: 10
layer_struct.start_nodes: 10, layer_struct.end_nodes: 10
layer_struct.start_nodes: 10, layer_struct.end_nodes: 1
Model:
MultiLayerNet(
  (layers): ModuleList(
    (0): Linear(in_features=1, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=1, bias=True)
  )
)
Current result for iteration 0 and batch 0 is 0.022669846192
Current result for iteration 0 and batch 1 is 0.315872460604
Current result for iteration 0 and batch 2 is 1.54319560528
Current result for iteration 0 and batch 3 is 1.0193721056
Current result for iteration 0 and batch 4 is 0.122840628028
Current result for iteration 0 and batch 5 is 0.772905647755
Current result for iteration 0 and batch 6 is 0.390398532152
Current result for iteration 0 and batch 7 is 0.00615305965766
Current result for iteration 0 and batch 8 is 0.125222340226
Current result for iteration 0 and batch 9 is 0.28028

Current result for iteration 0 and batch 144 is 0.288274019957
Current result for iteration 0 and batch 145 is 0.146096631885
Current result for iteration 0 and batch 146 is 0.517241179943
Current result for iteration 0 and batch 147 is 1.56864416599
Current result for iteration 0 and batch 148 is 0.22673086822
Current result for iteration 0 and batch 149 is 0.351059168577
Current result for iteration 0 and batch 150 is 1.57131874561
Current result for iteration 0 and batch 151 is 0.456995815039
Current result for iteration 0 and batch 152 is 0.27154314518
Current result for iteration 0 and batch 153 is 0.208084285259
Current result for iteration 0 and batch 154 is 1.48586916924
Current result for iteration 0 and batch 155 is 0.472102224827
Current result for iteration 0 and batch 156 is 1.16243577003
Current result for iteration 0 and batch 157 is 0.239809319377
Current result for iteration 0 and batch 158 is 0.0446500070393
Current result for iteration 0 and batch 159 is 1.6340025663

Current result for iteration 0 and batch 293 is 1.33465063572
Current result for iteration 0 and batch 294 is 1.12215399742
Current result for iteration 0 and batch 295 is 0.201306030154
Current result for iteration 0 and batch 296 is 0.490116894245
Current result for iteration 0 and batch 297 is 0.611869931221
Current result for iteration 0 and batch 298 is 0.155918776989
Current result for iteration 0 and batch 299 is 0.562040090561
Current result for iteration 0 and batch 300 is 0.0502307005227
Current result for iteration 0 and batch 301 is 1.41408848763
Current result for iteration 0 and batch 302 is 0.43136459589
Current result for iteration 0 and batch 303 is 0.238047301769
Current result for iteration 0 and batch 304 is 0.419429987669
Current result for iteration 0 and batch 305 is 0.0443297699094
Current result for iteration 0 and batch 306 is 1.52856826782
Current result for iteration 0 and batch 307 is 0.00490632001311
Current result for iteration 0 and batch 308 is 0.312068

Current result for iteration 0 and batch 429 is 0.584521532059
Current result for iteration 0 and batch 430 is 0.235734716058
Current result for iteration 0 and batch 431 is 0.522634804249
Current result for iteration 0 and batch 432 is 0.0138807399198
Current result for iteration 0 and batch 433 is 0.37102457881
Current result for iteration 0 and batch 434 is 0.588771164417
Current result for iteration 0 and batch 435 is 0.577247619629
Current result for iteration 0 and batch 436 is 0.0232120230794
Current result for iteration 0 and batch 437 is 1.43706130981
Current result for iteration 0 and batch 438 is 0.576133489609
Current result for iteration 0 and batch 439 is 0.0191570520401
Current result for iteration 0 and batch 440 is 0.202723667026
Current result for iteration 0 and batch 441 is 0.620134532452
Current result for iteration 0 and batch 442 is 0.152594923973
Current result for iteration 0 and batch 443 is 1.39802205563
Current result for iteration 0 and batch 444 is 1.34635

Current result for iteration 0 and batch 574 is 1.19467282295
Current result for iteration 0 and batch 575 is 1.58842384815
Current result for iteration 0 and batch 576 is 0.78411257267
Current result for iteration 0 and batch 577 is 0.530947983265
Current result for iteration 0 and batch 578 is 0.345094114542
Current result for iteration 0 and batch 579 is 0.158977180719
Current result for iteration 0 and batch 580 is 0.512741804123
Current result for iteration 0 and batch 581 is 1.47801840305
Current result for iteration 0 and batch 582 is 0.133093237877
Current result for iteration 0 and batch 583 is 0.00395915471017
Current result for iteration 0 and batch 584 is 0.301133871078
Current result for iteration 0 and batch 585 is 1.42787837982
Current result for iteration 0 and batch 586 is 1.50392830372
Current result for iteration 0 and batch 587 is 1.49310255051
Current result for iteration 0 and batch 588 is 0.652800738811
Current result for iteration 0 and batch 589 is 0.1589618027

...

Current result for iteration 200 and batch 597 is 0.658374667168
Current result for iteration 200 and batch 598 is 1.20109367371
Current result for iteration 200 and batch 599 is 0.0175326224416
Current result for iteration 200 and batch 600 is 0.981398642063
Current result for iteration 200 and batch 601 is 0.538347840309
Current result for iteration 200 and batch 602 is 0.492464572191
Current result for iteration 200 and batch 603 is 0.637429773808
Current result for iteration 200 and batch 604 is 0.104993730783
Current result for iteration 200 and batch 605 is 0.604865968227
Current result for iteration 200 and batch 606 is 0.564342737198
Current result for iteration 200 and batch 607 is 0.786901831627
Current result for iteration 200 and batch 608 is 0.850286364555
Current result for iteration 200 and batch 609 is 0.385779291391
Current result for iteration 200 and batch 610 is 1.38472270966
Current result for iteration 200 and batch 611 is 0.046838644892
Current result for iterati

Current result for iteration 250 and batch 158 is 0.0655006095767
Current result for iteration 250 and batch 159 is 0.868927776814
Current result for iteration 250 and batch 160 is 0.563303530216
Current result for iteration 250 and batch 161 is 0.0329677611589
Current result for iteration 250 and batch 162 is 0.435319662094
Current result for iteration 250 and batch 163 is 0.00895182602108
Current result for iteration 250 and batch 164 is 0.0214695222676
Current result for iteration 250 and batch 165 is 4.77726189274e-06
Current result for iteration 250 and batch 166 is 0.751017093658
Current result for iteration 250 and batch 167 is 0.249348178506
Current result for iteration 250 and batch 168 is 1.41007363796
Current result for iteration 250 and batch 169 is 0.141400709748
Current result for iteration 250 and batch 170 is 0.353291153908
Current result for iteration 250 and batch 171 is 0.0298832412809
Current result for iteration 250 and batch 172 is 0.181091114879
Current result fo

Current result for iteration 250 and batch 310 is 0.450481325388
Current result for iteration 250 and batch 311 is 0.725395858288
Current result for iteration 250 and batch 312 is 0.1728951931
Current result for iteration 250 and batch 313 is 0.000529709097464
Current result for iteration 250 and batch 314 is 0.400107890368
Current result for iteration 250 and batch 315 is 1.14009916782
Current result for iteration 250 and batch 316 is 0.0620783567429
Current result for iteration 250 and batch 317 is 0.172545447946
Current result for iteration 250 and batch 318 is 0.271679878235
Current result for iteration 250 and batch 319 is 0.00584531435743
Current result for iteration 250 and batch 320 is 0.170759379864
Current result for iteration 250 and batch 321 is 1.49472916126
Current result for iteration 250 and batch 322 is 0.584372639656
Current result for iteration 250 and batch 323 is 0.825922191143
Current result for iteration 250 and batch 324 is 0.0853716954589
Current result for ite

Current result for iteration 250 and batch 460 is 0.371565669775
Current result for iteration 250 and batch 461 is 0.00083962851204
Current result for iteration 250 and batch 462 is 0.0865077748895
Current result for iteration 250 and batch 463 is 0.579412043095
Current result for iteration 250 and batch 464 is 0.356553196907
Current result for iteration 250 and batch 465 is 0.533247053623
Current result for iteration 250 and batch 466 is 0.437150359154
Current result for iteration 250 and batch 467 is 0.194221839309
Current result for iteration 250 and batch 468 is 0.411780595779
Current result for iteration 250 and batch 469 is 0.0124338809401
Current result for iteration 250 and batch 470 is 0.69098430872
Current result for iteration 250 and batch 471 is 0.00373421772383
Current result for iteration 250 and batch 472 is 0.935406029224
Current result for iteration 250 and batch 473 is 0.644339025021
Current result for iteration 250 and batch 474 is 1.29638659954
Current result for it

Current result for iteration 250 and batch 586 is 1.1939214468
Current result for iteration 250 and batch 587 is 0.8169683218
Current result for iteration 250 and batch 588 is 0.484487921
Current result for iteration 250 and batch 589 is 0.432870715857
Current result for iteration 250 and batch 590 is 0.0492384396493
Current result for iteration 250 and batch 591 is 0.00311593594961
Current result for iteration 250 and batch 592 is 0.601512312889
Current result for iteration 250 and batch 593 is 0.062781393528
Current result for iteration 250 and batch 594 is 0.550951659679
Current result for iteration 250 and batch 595 is 0.188342258334
Current result for iteration 250 and batch 596 is 0.15998762846
Current result for iteration 250 and batch 597 is 0.654132127762
Current result for iteration 250 and batch 598 is 1.23802375793
Current result for iteration 250 and batch 599 is 0.0174015834928
Current result for iteration 250 and batch 600 is 0.970073342323
Current result for iteration 2

In [8]:
v = [None] * 10
num_v = len(v)

In [6]:
for i in range(num_v - 1):
    print("v[{0}] = {1}".format(i, v[i]))
print("v[{0}] = {1}".format(num_v, v[num_v]))
    

v[0] = None
v[1] = None
v[2] = None
v[3] = None
v[4] = None
v[5] = None
v[6] = None
v[7] = None
v[8] = None


IndexError: list index out of range

In [14]:
torch.manual_seed(1)    # reproducible

BATCH_SIZE = 5
# BATCH_SIZE = 8

x = torch.linspace(1, 10, 10)       # this is x data (torch tensor)
y = torch.linspace(10, 1, 10)       # this is y data (torch tensor)

torch_dataset = Data.TensorDataset(x, y)
loader = Data.DataLoader(
    dataset=torch_dataset,      # torch TensorDataset format
    batch_size=BATCH_SIZE,      # mini batch size
    shuffle=True,               # random shuffle for training
    num_workers=2,              # subprocesses for loading data
)


def show_batch():
    for epoch in range(3):   # train entire dataset 3 times
        for step, (batch_x, batch_y) in enumerate(loader):  # for each training step
            # train your data...
            print('Epoch: ', epoch, '| Step: ', step, '| batch x: ',
                  batch_x.numpy(), '| batch y: ', batch_y.numpy())

In [15]:
show_batch()

('Epoch: ', 0, '| Step: ', 0, '| batch x: ', array([ 5.,  7., 10.,  3.,  4.], dtype=float32), '| batch y: ', array([6., 4., 1., 8., 7.], dtype=float32))
('Epoch: ', 0, '| Step: ', 1, '| batch x: ', array([2., 1., 8., 9., 6.], dtype=float32), '| batch y: ', array([ 9., 10.,  3.,  2.,  5.], dtype=float32))
('Epoch: ', 1, '| Step: ', 0, '| batch x: ', array([ 4.,  6.,  7., 10.,  8.], dtype=float32), '| batch y: ', array([7., 5., 4., 1., 3.], dtype=float32))
('Epoch: ', 1, '| Step: ', 1, '| batch x: ', array([5., 3., 2., 1., 9.], dtype=float32), '| batch y: ', array([ 6.,  8.,  9., 10.,  2.], dtype=float32))
('Epoch: ', 2, '| Step: ', 0, '| batch x: ', array([ 4.,  2.,  5.,  6., 10.], dtype=float32), '| batch y: ', array([7., 9., 6., 5., 1.], dtype=float32))
('Epoch: ', 2, '| Step: ', 1, '| batch x: ', array([3., 9., 1., 8., 7.], dtype=float32), '| batch y: ', array([ 8.,  2., 10.,  3.,  4.], dtype=float32))


In [60]:
torch.nn.Module??

In [36]:
torch.randn(10, 2)

tensor([[ 0.9869, -0.1750],
        [-1.5581, -0.9386],
        [ 2.0125, -2.6450],
        [ 1.1887,  0.8944],
        [ 1.4478, -0.6183],
        [ 0.2341,  0.1839],
        [-0.1306,  0.7640],
        [ 1.9316, -1.6759],
        [-0.6829,  0.0820],
        [-0.0792, -0.4549]])